In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import IPython

In [ ]:
pd.set_option('max_columns' , None) #to see all the columns
df = pd.read_csv(r'C:\Users\Chitwan\Downloads\summeranalytics2020\train.csv')
df_test = pd.read_csv(r'C:\Users\Chitwan\Downloads\summeranalytics2020\test.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
X = df.drop(["Attrition"] , axis = 1)

In [ ]:
y = df["Attrition"]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 1)

In [ ]:
#Now we'll check the categorical variables so as to determine whether to go for label encoding or One hot encoding.
obj_dict_train = { obj_col : list(X_train[obj_col].unique()) for obj_col in X_train.select_dtypes(include = 'object')}

In [ ]:
obj_dict_train

In [ ]:
obj_dict_test = {obj_col : list(X_test[obj_col].unique()) for obj_col in X_test.select_dtypes(include = 'object')}

In [ ]:
obj_dict_test #checking wether the training set and testing set have the same categorical varaibles

In [ ]:
label_col = ["BusinessTravel" , "OverTime"] #Ordinal Categorical Data

In [ ]:
OH_X_train = list(set(df.select_dtypes(include = "object")) - set(label_col))

In [ ]:
OH_X_train

In [ ]:
#Now we'll begin with One Hot Encoding
X_train = pd.concat([X_train , pd.get_dummies(X_train[OH_X_train])] , axis = 1)
X_train.drop(OH_X_train, axis=1, inplace=True)

In [ ]:
X_train.head()

In [ ]:
X_test = pd.concat([X_test , pd.get_dummies(X_test[OH_X_train])] , axis = 1)

In [ ]:
X_test.drop(OH_X_train, axis = 1, inplace = True)

In [ ]:
X_test.head()

In [ ]:
OH_X_test = list(set(df_test.select_dtypes(include = 'object')) - set(label_col))

In [ ]:
df_test = pd.concat([df_test , pd.get_dummies(df_test[OH_X_test])] , axis = 1)

In [ ]:
df_test.drop(OH_X_test , axis = 1 , inplace =True)

In [ ]:
df_test.head()

In [ ]:
# Now we have to encode ordinal categorical variables
# Business Travel
cat1 = pd.Categorical(X_train.BusinessTravel , categories = ['Non-Travel' , 'Travel_Rarely' , 'Travel_Frequently'] , ordered = True)
labels1 , unique1 = pd.factorize(cat1 , sort =True)
X_train['BusinessTravel'] = labels1
# OverTime
cat2 = pd.Categorical(X_train.OverTime , categories = ['No' , 'Yes'] , ordered = True)
labels2 , unique2 = pd.factorize(cat2 , sort =True)
X_train['OverTime'] = labels2

In [ ]:
cat3 = pd.Categorical(X_test.BusinessTravel , categories = ['Non-Travel' , 'Travel_Rarely' , 'Travel_Frequently'] , ordered = True)
labels3 , unique3 = pd.factorize(cat3 , sort =True)
X_test['BusinessTravel'] = labels3
# OverTime
cat4 = pd.Categorical(X_test.OverTime , categories = ['No' , 'Yes'] , ordered = True)
labels4 , unique4 = pd.factorize(cat4 , sort =True)
X_test['OverTime'] = labels4

In [ ]:
#Encoding Test file
#Business Travel
cat_test = pd.Categorical(df_test.BusinessTravel , categories = ['Non-Travel' , 'Travel_Rarely' , 'Travel_Frequently'] , ordered = True)
labels_test , unique_test = pd.factorize(cat_test , sort =True)
df_test['BusinessTravel'] = labels_test
#OverTime
cat_test_1 = pd.Categorical(df_test.OverTime , categories = ['No' , 'Yes'] , ordered = True)
labels_test_1 , unique_test_1 = pd.factorize(cat_test_1 , sort =True)
df_test['OverTime'] = labels_test_1

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
df_test.drop(['Behaviour' , 'Gender_Female'] , axis = 1 , inplace = True)

In [ ]:
df_test.head()

In [ ]:
X_train.nunique()

In [ ]:
#Feature Scaling
num_cols = [ col for col in X_train.columns if X_train[col].nunique() > 5]

In [ ]:
from sklearn.preprocessing import StandardScaler
num_scaler = StandardScaler(copy = False)
scaled_train = pd.DataFrame(num_scaler.fit_transform(X_train[num_cols]))
scaled_test = pd.DataFrame(num_scaler.fit_transform(X_test[num_cols]))
scaled_df_test = pd.DataFrame(num_scaler.fit_transform(df_test[num_cols]))
scaled_train.columns = num_cols
scaled_test.columns = num_cols
scaled_df_test.columns = num_cols
scaled_train.index = X_train.index
scaled_test.index = X_test.index
scaled_df_test.index = df_test.index


In [ ]:
scaled_train.head()

In [ ]:
X_train.drop(num_cols , axis = 1 , inplace = True)

In [ ]:
X_train.head()

In [ ]:
X_train = pd.concat([X_train , scaled_train] , axis = 1 )
X_test = pd.concat([X_test , scaled_test] , axis = 1)
df_test = pd.concat([df_test, scaled_df_test] , axis = 1)

In [ ]:
df_test.head()

In [ ]:
plt.style.use('ggplot')
sns.set_style('white')

In [ ]:
data_count = pd.concat([X_train.drop(columns = num_cols) , y] , axis = 1)

In [ ]:
len(data_count.columns)

In [ ]:
fig , axes = plt.subplots(round((len(data_count.columns)-1)/3) , 3 , figsize = (12,25))
for i, ax in enumerate(fig.axes):
    if i <= len(data_count.columns):
     ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=45)
     sns.countplot(x=data_count.columns[i], hue=data_count.Attrition, alpha=0.7, data=data_count, ax=ax)

fig.tight_layout()

In [ ]:
#Numerical Data
X_train[num_cols].hist(figsize = (14,20) ,  xlabelsize = 8 , ylabelsize = 8);

In [ ]:
#prediction

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(n_estimators = 500)

In [ ]:
model.fit(X_train , y_train)

In [ ]:
preds = model.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score,accuracy_score

In [ ]:
roc_auc_score(y_test,preds)

In [ ]:
df_test.shape

In [ ]:
X_train.shape

In [ ]:
X_train.head()

In [ ]:
final_pred = model.predict(df_test)

In [ ]:
final_pred

In [ ]:
output = pd.concat([df_test.Id,pd.Series(final_pred)], axis=1)
output.columns=['Id','Id2','Attrition']
output.set_index('Id' , inplace = True)

In [ ]:
output.drop(columns = 'Id2' ,axis = 1 ,inplace = True)

In [ ]:
output.to_csv("submission.csv",index=False)

In [ ]:
output